# Unit test

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
from skforecast.metrics import root_mean_squared_scaled_error
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries


# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y_feature_selection
from skforecast.model_selection.tests.fixtures_model_selection import exog_feature_selection


metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
#metrics = [add_y_train_argument(metric) for metric in metrics]
from skforecast.datasets import fetch_dataset



In [3]:
data = fetch_dataset(name="items_sales", verbose=False)
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
data_test = data.loc[end_train:, :].copy()
levels = ["item_1", "item_2", "item_3"]
exog_features = None

metrics = [
    "mean_absolute_error",
    "mean_squared_error",
    mean_absolute_percentage_error,
    mean_absolute_scaled_error,
    root_mean_squared_scaled_error,
]
steps = 1
initial_train_size = 100
param_grid = {
    "alpha": np.logspace(-3, 3, 1),
}
lags_grid = [3]
param_grid = list(ParameterGrid(param_grid))

forecaster = ForecasterAutoregMultiSeries(
    regressor=Ridge(random_state=678),
    lags=3,
    transformer_series=StandardScaler(),
)

# forecaster = ForecasterAutoregMultiVariate(
#     regressor=Ridge(random_state=678),
#     lags=3,
#     steps=1,
#     level="item_1",
#     # transformer_series=StandardScaler(),
#     # transformer_exog=StandardScaler(),
# )

X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
    forecaster._train_test_split_one_step_ahead(
        series=data.loc[:, levels],
        exog=data.loc[:, exog_features] if exog_features else None,
        initial_train_size=initial_train_size,
    )
)

display(data)
display(X_train)
display(X_test)
display(y_train)
display(y_test)
display(X_train_encoding)
display(X_test_encoding)


results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        return_best        = False,
        verbose            = False,
        show_progress      = True
    )

results_one_step_ahead

,item_1,item_2,item_3
date,,,
2012-01-01,8.253175,21.047727,19.429739
2012-01-02,22.777826,26.578125,28.009863
2012-01-03,27.549099,31.751042,32.078922
2012-01-04,25.895533,24.567708,27.252276
2012-01-05,21.379238,18.191667,20.357737
...,...,...,...
2014-12-28,17.329233,18.189583,20.586030
2014-12-29,19.611623,24.539583,28.127390
2014-12-30,18.857026,17.677083,21.555782


,lag_1,lag_2,lag_3,_level_skforecast
date,,,,
2012-01-04,1.736038,0.069055,-5.005556,0
2012-01-05,1.158317,1.736038,0.069055,0
2012-01-06,-0.419583,1.158317,1.736038,0
2012-01-07,-0.514822,-0.419583,1.158317,0
2012-01-08,-0.714936,-0.514822,-0.419583,0
...,...,...,...,...
2012-04-05,-0.338919,-0.047092,-1.404941,2
2012-04-06,-0.356970,-0.338919,-0.047092,2
2012-04-07,0.121229,-0.356970,-0.338919,2


,lag_1,lag_2,lag_3,_level_skforecast
date,,,,
2012-04-10,-1.600750,-1.602861,-1.378975,0
2012-04-11,-0.666965,-1.600750,-1.602861,0
2012-04-12,0.154228,-0.666965,-1.600750,0
2012-04-13,0.035007,0.154228,-0.666965,0
2012-04-14,-0.048676,0.035007,0.154228,0
...,...,...,...,...
2014-12-28,0.082209,-0.455532,-1.455941,2
2014-12-29,-0.186787,0.082209,-0.455532,2
2014-12-30,1.573370,-0.186787,0.082209,2


date
2012-01-04    1.158317
2012-01-05   -0.419583
2012-01-06   -0.514822
2012-01-07   -0.714936
2012-01-08   -0.877238
                ...   
2012-04-05   -0.356970
2012-04-06    0.121229
2012-04-07   -0.422595
2012-04-08   -0.144628
2012-04-09   -1.310729
Name: y, Length: 291, dtype: float64

date
2012-04-10   -0.666965
2012-04-11    0.154228
2012-04-12    0.035007
2012-04-13   -0.048676
2012-04-14   -0.464676
                ...   
2014-12-28   -0.186787
2014-12-29    1.573370
2014-12-30    0.039553
2014-12-31   -0.649055
2015-01-01   -0.336703
Name: y, Length: 2991, dtype: float64

...

date
2012-04-10    item_1
2012-04-11    item_1
2012-04-12    item_1
2012-04-13    item_1
2012-04-14    item_1
               ...  
2014-12-28    item_3
2014-12-29    item_3
2014-12-30    item_3
2014-12-31    item_3
2015-01-01    item_3
Length: 2991, dtype: object

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection_multiseries\model_selection_multiseries.py:1747: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

,levels,lags,lags_label,params,mean_absolute_error__weighted_average,mean_absolute_error__average,mean_absolute_error__pooling,mean_squared_error__weighted_average,mean_squared_error__average,mean_squared_error__pooling,mean_absolute_percentage_error__weighted_average,mean_absolute_percentage_error__average,mean_absolute_percentage_error__pooling,mean_absolute_scaled_error__weighted_average,mean_absolute_scaled_error__average,mean_absolute_scaled_error__pooling,root_mean_squared_scaled_error__weighted_average,root_mean_squared_scaled_error__average,root_mean_squared_scaled_error__pooling,alpha
0,"[item_1, item_2, item_3]","[1, 2, 3]","[1, 2, 3]",{'alpha': 0.001},3.265788,3.265788,3.265788,17.715033,17.715033,17.715033,0.235861,0.235861,0.235861,1.208495,1.208495,1.230248,1.104972,1.104972,1.133532,0.001


In [5]:
np.logspace(-1, 1, 3)

array([ 0.1, 10. ])

In [9]:
data = fetch_dataset(name="items_sales", verbose=False)
end_train = '2014-07-15 23:59:00'
initial_train_size = len(data.loc[:end_train, :].copy())
data_test  = data.loc[end_train:, :].copy()
levels = ['item_1', 'item_2', 'item_3']
exog_features = None


def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 100
    param_grid = {
        "alpha": np.logspace(-1, 1, 3),
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='item_1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

0


c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection_multiseries\model_selection_multiseries.py:1747: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


1


c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection_multiseries\model_selection_multiseries.py:1747: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


2


KeyboardInterrupt: 

## _calculate_metrics_multiseries vs _calculate_metrics_multiseries_one_step_ahead

In [8]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [8]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

NameError: name 'fetch_dataset' is not defined

In [9]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal',
                 transformer_series = StandardScaler(),
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

X, y = forecaster.create_train_X_y(series=data)
X_train = X.loc[X.index <= end_train, :]
y_train = y.loc[y.index <= end_train]
X_test = X.loc[X.index > end_train, :]
y_test = y.loc[y.index > end_train]

forecaster.regressor.fit(X_train, y_train)
pred = forecaster.regressor.predict(X_test)

NameError: name 'LGBMRegressor' is not defined

In [10]:
# Test results _calculate_metrics_multiseries_one_step_ahead and _calculate_metrics_multiseries
# ==============================================================================
# Metrics should be equal when using step=1 in backtesting

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

if forecaster.encoding in ['ordinal', 'ordinal_category']:
    X_train_encoding = forecaster.encoder.inverse_transform(X_train[['_level_skforecast']]).ravel()
    X_test_encoding = forecaster.encoder.inverse_transform(X_test[['_level_skforecast']]).ravel()
elif forecaster.encoding == 'onehot':
    X_train_encoding = forecaster.encoder.inverse_transform(
                            X_train.loc[:, forecaster.encoding_mapping.keys()]
                        ).ravel()
    X_test_encoding = forecaster.encoder.inverse_transform(
                            X_test.loc[:, forecaster.encoding_mapping.keys()]
                        ).ravel()
X_train_encoding = pd.Series(X_train_encoding, index=X_train.index)
X_test_encoding = pd.Series(X_test_encoding, index=X_test.index)

metrics_one_step_ahead = _calculate_metrics_multiseries_one_step_ahead(
    y_true = y_test,
    y_pred = pred,
    y_pred_encoding = X_test_encoding,
    y_train = y_train,
    y_train_encoding = X_train_encoding,
    levels = ['item_1', 'item_2', 'item_3'],
    metrics = metrics,
    add_aggregated_metric = True
)

display(metrics_one_step_ahead)

# Metrics with _calculate_metrics_multiseries
folds = _create_backtesting_folds(
    data = data,
    window_size = 24,
    initial_train_size = len(data_train),
    test_size = 1,
    externally_fitted = False,
    refit = False,
    fixed_train_size = True,
    gap = 0,
    skip_folds = None,
    allow_incomplete_fold = True,
    return_all_indexes = False,
    differentiation = None,
    verbose = False
)
_, predictions = backtesting_forecaster_multiseries(
    series=data,
    forecaster=forecaster,
    steps=1,
    metric=metrics,
    initial_train_size = len(data_train),
    refit=False,
    add_aggregated_metric=True,
    show_progress=False
)
metrics_backtesting = _calculate_metrics_multiseries(
    series = data,
    predictions= predictions,
    folds= folds,
    span_index= data.index,
    window_size = 24,
    metrics= metrics,
    levels= ['item_1', 'item_2', 'item_3'],
    add_aggregated_metric = True
)

display(metrics_backtesting)

pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)

NameError: name 'forecaster' is not defined

## _evaluate_grid_hyperparameters_multiseries: backtesting vs one-step-ahead

In [11]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries
from skforecast.metrics import mean_absolute_scaled_error

In [12]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()
exog_features = None

forecaster = ForecasterAutoregMultiSeries(
    regressor          = LGBMRegressor(random_state=123, verbose=-1),
    lags               = 24,
    encoding           = 'ordinal',
    transformer_series = None,
    transformer_exog   = None,
    weight_func        = None,
    series_weights     = None,
    differentiation    = None,
    dropna_from_series = False,
    fit_kwargs         = None,
    forecaster_id      = None
)
lags_grid = [3, 10]
param_grid = {
    'n_estimators': [5, 10],
    'max_depth': [2, 3]
}
param_grid = list(ParameterGrid(param_grid))

results_backtesting = _evaluate_grid_hyperparameters_multiseries(
                            forecaster            = forecaster,
                            series                = data,
                            param_grid            = param_grid,
                            steps                 = 1,
                            metric                = metrics,
                            aggregate_metric      = ['weighted_average', 'average', 'pooling'],
                            initial_train_size    = len(data_train),
                            method                = 'backtesting',
                            fixed_train_size      = True,
                            gap                   = 0,
                            skip_folds            = None,
                            allow_incomplete_fold = True,
                            levels                = None,
                            exog                  = data.loc[:end_validation, exog_features] if exog_features else None,
                            lags_grid             = lags_grid,
                            refit                 = False,
                            n_jobs                = 'auto',
                            return_best           = False,
                            verbose               = False,
                            show_progress         = False,
                            suppress_warnings     = False
                        )

results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
                            forecaster            = forecaster,
                            series                = data,
                            param_grid            = param_grid,
                            metric                = metrics,
                            initial_train_size    = len(data_train),
                            method                = 'one_step_ahead',
                            exog                  = data.loc[:data_train, exog_features] if exog_features else None,
                            lags_grid             = lags_grid,
                            return_best           = False,
                            verbose               = False,
                            show_progress         = False
                        )

assert results_backtesting.equals(results_one_step_ahead)


NameError: name 'fetch_dataset' is not defined